# This is a single file that will run all relevant tests and output the corresponding results
### NOTE1: Cell 3 has a hard coded path to the imbd data to avoid downloading and unzipping in code, so please adjust that path such that it points to the correct aclImbd folder in your directory
### NOTE2: The final models for each model-data section are hardcoded with the parameters we found to work best (ie the GSCV results are not dynamically populating the final models' parameters)

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

##### Get the data

In [2]:
def getIMBD(path):
    posData = []
    posTarget = []
    negData = []
    negTarget = []
    i = 0
    directory = os.path.join(path, "pos")
    for filename in os.listdir(directory):
        f = open(os.path.join(directory, filename), "r", encoding="utf8")
        posData.append(f.read())
        posTarget.append(1)
        f.close()
        i = i + 1
    i = 0
    directory = os.path.join(path, "neg")
    for filename in os.listdir(directory):
        f = open(os.path.join(directory, filename), "r", encoding="utf8")
        negData.append(f.read())
        negTarget.append(0)
        f.close()
        i = i + 1
    return (posData + negData), (posTarget + negTarget)

In [3]:
twenty_train = fetch_20newsgroups(subset='train', remove=(['headers', 'footers', 'quotes']), shuffle=True)
twenty_test = fetch_20newsgroups(subset='test', remove=(['headers', 'footers', 'quotes']), shuffle=True)
imbd_train_data, imbd_train_target = getIMBD("..\\aclImdb\\train")
imbd_test_data, imbd_test_target = getIMBD("..\\aclImdb\\test")

# Logistic Regression - Twenty News Group
##### Set up a pre-processing pipeline separately from the main-pipeline so that later we can pre-process before training our final model on the train dataset.  Define the parameters over which to do the GSCV and then perform the GSCV on the set of permutations of those parameters.

In [4]:
twentyLogRegPreProcessingPipe = Pipeline([
    ('vect', CountVectorizer(lowercase=True, max_df=0.3, max_features=100000, min_df=3, ngram_range=(1, 1), stop_words='english', strip_accents='ascii')),
    ('tfidf', TfidfTransformer()),
    ('scale', StandardScaler(with_mean=False)),
    ('norm', Normalizer())
])
twentyLogRegPipe = Pipeline([
    ('ppp', twentyLogRegPreProcessingPipe),
    ('clf', LogisticRegression())
])
twentyLogRegParam = {
    'clf__penalty': ['l2'],
    'clf__dual': [False, True],
    'clf__tol': [0.00001, 0.0001, 0.001],
    'clf__C': [0.1, 1.0, 2.5, 10.0],
    'clf__fit_intercept': [False, True],
    'clf__intercept_scaling': [0.1, 1.0, 2.0],
    'clf__class_weight': [None, 'balanced'],
    'clf__random_state': [None],
    'clf__solver': ['lbfgs', 'sag', 'saga'],
    'clf__max_iter': [100, 1000],
    'clf__multi_class': ['auto'],
    'clf__l1_ratio': [None] #Pointless cause not using liblinear nor elastic-net
}
twentyLogRegGSCV = GridSearchCV(twentyLogRegPipe, twentyLogRegParam, cv=10, n_jobs=-1, verbose=1000)
twentyLogRegGSCV = twentyLogRegGSCV.fit(twenty_train.data, twenty_train.target)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elaps

Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed:   39.2s
Pickling array (shape=(11314,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   39.3s
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:   41.6s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   42.2s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  43 tasks      | elapsed:   42.8s
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapse

[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:  1.4min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  1.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  83 tasks      | elapsed:  1.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:  1.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Para

Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 123 tasks      | elapsed:  2.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  2.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 125 tasks      | elapsed:  2.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 126 tasks      | elapsed:  2.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 127 tasks      | elapsed:  2.2min
Pickl

Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  3.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed:  3.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 167 tasks      | elapsed:  3.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  3.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 169 tasks      | elapsed:  3.1min
Pickl

...

[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 29.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1247 tasks      | elapsed: 30.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1248 tasks      | elapsed: 30.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1249 tasks      | elapsed: 30.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1250 tasks      | elapsed: 30.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).


[Parallel(n_jobs=-1)]: Done 1287 tasks      | elapsed: 31.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1288 tasks      | elapsed: 31.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1289 tasks      | elapsed: 31.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1290 tasks      | elapsed: 31.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1291 tasks      | elapsed: 31.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).


[Parallel(n_jobs=-1)]: Done 1328 tasks      | elapsed: 32.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1329 tasks      | elapsed: 33.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1330 tasks      | elapsed: 33.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1331 tasks      | elapsed: 33.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1332 tasks      | elapsed: 33.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).


[Parallel(n_jobs=-1)]: Done 1369 tasks      | elapsed: 34.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1370 tasks      | elapsed: 34.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1371 tasks      | elapsed: 34.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1372 tasks      | elapsed: 34.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1373 tasks      | elapsed: 34.4min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).


[Parallel(n_jobs=-1)]: Done 1410 tasks      | elapsed: 36.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1411 tasks      | elapsed: 36.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1412 tasks      | elapsed: 36.2min
[Parallel(n_jobs=-1)]: Done 1413 tasks      | elapsed: 36.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1414 tasks      | elapsed: 36.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).


##### The best score and associated parameters followed by a table of more detailed results

In [5]:
twentyLogRegGSCV.best_score_

0.7498690142561227

In [6]:
for param_name in sorted(twentyLogRegParam.keys()):
    print("%s: %r" % (param_name, twentyLogRegGSCV.best_params_[param_name]))

clf__C: 1.0
clf__class_weight: 'balanced'
clf__dual: False
clf__fit_intercept: False
clf__intercept_scaling: 1.0
clf__l1_ratio: None
clf__max_iter: 100
clf__multi_class: 'auto'
clf__penalty: 'l2'
clf__random_state: None
clf__solver: 'lbfgs'
clf__tol: 1e-05


In [7]:
df = pd.DataFrame(twentyLogRegGSCV.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__C,param_clf__class_weight,param_clf__dual,param_clf__fit_intercept,param_clf__intercept_scaling,param_clf__l1_ratio,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,3.402288,0.151072,0.232457,0.043210,0.1,None,False,False,1,None,...,0.739399,0.749779,0.726790,0.748011,0.735632,0.758621,0.725906,0.737142,0.013564,91
1,3.134807,0.179347,0.230609,0.044258,0.1,None,False,False,1,None,...,0.739399,0.749779,0.726790,0.748011,0.735632,0.758621,0.725906,0.737142,0.013564,91
2,3.223356,0.133195,0.235923,0.039754,0.1,None,False,False,1,None,...,0.739399,0.749779,0.726790,0.748011,0.735632,0.758621,0.725906,0.737142,0.013564,91
3,3.969146,0.228706,0.204403,0.025236,0.1,None,False,False,1,None,...,0.739399,0.749779,0.726790,0.748011,0.735632,0.758621,0.725906,0.737142,0.013564,91
4,3.512706,0.162748,0.198833,0.024325,0.1,None,False,False,1,None,...,0.739399,0.749779,0.726790,0.748011,0.735632,0.758621,0.725906,0.737142,0.013564,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,12.079623,6.447649,0.206249,0.022963,10,balanced,False,False,1,None,...,0.737633,0.767462,0.715296,0.752431,0.723254,0.746242,0.725022,0.735727,0.016114,110
140,8.411997,2.046037,0.201561,0.021481,10,balanced,False,False,1,None,...,0.737633,0.767462,0.715296,0.752431,0.723254,0.746242,0.725022,0.735727,0.016114,110
141,13.409557,2.167023,0.185853,0.013072,10,balanced,False,False,1,None,...,0.737633,0.767462,0.715296,0.752431,0.723254,0.746242,0.725022,0.735727,0.016114,110
142,9.756076,0.736460,0.186768,0.014080,10,balanced,False,False,1,None,...,0.737633,0.767462,0.715296,0.752431,0.723254,0.746242,0.725022,0.735727,0.016114,110


##### Use the pre-processing pipe to pre-process both the train and test data separately, then fit a model with the above best parameters to the train data and evaluate the result on the test data

In [45]:
twentyLogRegPreProcessingPipe.fit(twenty_train.data)
X_train_LogReg_twenty = twentyLogRegPreProcessingPipe.transform(twenty_train.data)
X_test_LogReg_twenty = twentyLogRegPreProcessingPipe.transform(twenty_test.data)
twentyLogRegDefault = LogisticRegression()
twentyLogRegDefault.fit(X_train_LogReg_twenty, twenty_train.target)
y_pred_LogReg_twenty_Default = twentyLogRegDefault.predict(X_test_LogReg_twenty)
metrics.accuracy_score(twenty_test.target, y_pred_LogReg_twenty_Default)

0.6820233669676049

In [46]:
twentyLogReg = LogisticRegression(C=1.0, class_weight='balanced', dual=False, fit_intercept=False, intercept_scaling=1.0, l1_ratio=None, max_iter=100, multi_class='auto', penalty='l2', random_state=None, solver='lbfgs', tol=1e-05)
twentyLogReg.fit(X_train_LogReg_twenty, twenty_train.target)
y_pred_LogReg_twenty = twentyLogReg.predict(X_test_LogReg_twenty)
metrics.accuracy_score(twenty_test.target, y_pred_LogReg_twenty)

0.6776420605416887

In [47]:
metrics.confusion_matrix(twenty_test.target, y_pred_LogReg_twenty)

array([[158,   3,   1,   1,   1,   1,   3,   2,   3,   3,   4,   4,   3,
          3,   5,  58,  15,   9,   7,  35],
       [ 10, 269,  19,  14,   8,  21,   9,   2,   4,   2,   0,  14,   4,
          0,   4,   3,   0,   1,   1,   4],
       [ 20,  23, 235,  35,  14,  18,   5,   2,   5,   0,   0,   3,   3,
          2,   9,   7,   4,   0,   6,   3],
       [  8,  13,  32, 253,  24,   9,  17,   4,   1,   0,   1,   3,  21,
          2,   1,   2,   0,   1,   0,   0],
       [ 16,  10,  14,  22, 260,   8,  14,   3,   2,   1,   0,   4,  12,
          3,   5,   2,   3,   1,   4,   1],
       [  9,  40,  24,  10,   3, 280,   4,   2,   0,   3,   2,   5,   3,
          1,   3,   3,   1,   0,   2,   0],
       [  9,   2,   4,  24,  18,   0, 290,  11,   5,   2,   3,   3,   6,
          0,   4,   2,   2,   0,   3,   2],
       [ 27,   5,   5,   1,   6,   2,  14, 271,  13,   0,   2,   3,  10,
          3,   6,   1,   6,   4,  13,   4],
       [ 21,   3,   1,   3,   0,   1,   6,  31, 272,   3,   5,  

In [48]:
print(metrics.classification_report(twenty_test.target, y_pred_LogReg_twenty, target_names=twenty_test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.29      0.50      0.37       319
           comp.graphics       0.66      0.69      0.68       389
 comp.os.ms-windows.misc       0.64      0.60      0.62       394
comp.sys.ibm.pc.hardware       0.63      0.65      0.64       392
   comp.sys.mac.hardware       0.74      0.68      0.71       385
          comp.windows.x       0.79      0.71      0.75       395
            misc.forsale       0.72      0.74      0.73       390
               rec.autos       0.73      0.68      0.70       396
         rec.motorcycles       0.83      0.68      0.75       398
      rec.sport.baseball       0.91      0.81      0.86       397
        rec.sport.hockey       0.90      0.90      0.90       399
               sci.crypt       0.75      0.69      0.72       396
         sci.electronics       0.68      0.56      0.61       393
                 sci.med       0.80      0.76      0.78       396
         

# Logistic Regression - IBMD Movie Reviews
##### Set up a pre-processing pipeline separately from the main-pipeline so that later we can pre-process before training our final model on the train dataset.  Define the parameters over which to do the GSCV and then perform the GSCV on the set of permutations of those parameters.

In [11]:
imbdLogRegPreProcessingPipe = Pipeline([
    ('vect', CountVectorizer(lowercase=True, max_df=0.4, max_features=10000, min_df=3, ngram_range=(1, 3), stop_words='english', strip_accents='ascii')),
    ('tfidf', TfidfTransformer()),
    ('scale', StandardScaler(with_mean=False)),
    ('norm', Normalizer())
])
imbdLogRegPipe = Pipeline([
    ('ppp', imbdLogRegPreProcessingPipe),
    ('clf', LogisticRegression())
])
imbdLogRegParam = {
    'clf__penalty': ['l2'],
    'clf__dual': [False],
    'clf__tol': [0.00001, 0.0001, 0.001],
    'clf__C': [0.1, 1.0, 2.5, 10.0],
    'clf__fit_intercept': [False],
    'clf__intercept_scaling': [1.0], #Pointless cause fit_intercept is false
    'clf__class_weight': [None, 'balanced'],
    'clf__random_state': [None],
    'clf__solver': ['lbfgs', 'sag', 'saga'],
    'clf__max_iter': [100, 1000],
    'clf__multi_class': ['auto'],
    'clf__l1_ratio': [None] #Pointless cause not using liblinear nor elastic-net
}
imbdLogRegGSCV = GridSearchCV(imbdLogRegPipe, imbdLogRegParam, cv=10, n_jobs=-1, verbose=1000)
imbdLogRegGSCV = imbdLogRegGSCV.fit(imbd_train_data, imbd_train_target)

Fitting 10 folds for each of 144 candidates, totalling 1440 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   30.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   31.3s
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   4 task

[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed:  7.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:  7.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  7.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:  7.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  55 tasks      | elapsed:  7.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  7.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  7.7min
Pickling array (shap

[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 14.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 106 tasks      | elapsed: 14.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 107 tasks      | elapsed: 14.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed: 14.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 15.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 110 tasks      | elapsed: 15.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 111 tasks      | elapsed: 15.1min
Pickling array (shap

[Parallel(n_jobs=-1)]: Done 159 tasks      | elapsed: 21.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed: 21.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 161 tasks      | elapsed: 21.8min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed: 22.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 163 tasks      | elapsed: 22.2min
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed: 22.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 22.3min
Pickling array (shap

[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 28.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 214 tasks      | elapsed: 28.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 215 tasks      | elapsed: 29.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed: 29.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed: 29.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 218 tasks      | elapsed: 29.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 219 tasks      | elapsed: 29.5min
Pickling array (shap

...

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1180 tasks      | elapsed: 157.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1181 tasks      | elapsed: 157.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1182 tasks      | elapsed: 157.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1183 tasks      | elapsed: 157.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1184 tasks      | elapsed: 157.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1185 tasks      | elapsed: 157.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1233 tasks      | elapsed: 164.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 164.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1235 tasks      | elapsed: 164.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1236 tasks      | elapsed: 164.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1237 tasks      | elapsed: 165.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1238 tasks      | elapsed: 165.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1286 tasks      | elapsed: 171.8min
[Parallel(n_jobs=-1)]: Done 1287 tasks      | elapsed: 171.8min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1288 tasks      | elapsed: 171.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1289 tasks      | elapsed: 172.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1290 tasks      | elapsed: 172.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1291 tasks      | elapsed: 172.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1339 tasks      | elapsed: 179.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1340 tasks      | elapsed: 179.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1341 tasks      | elapsed: 179.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1342 tasks      | elapsed: 179.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1343 tasks      | elapsed: 179.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1344 tasks      | elapsed: 179.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1392 tasks      | elapsed: 186.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1393 tasks      | elapsed: 186.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1394 tasks      | elapsed: 186.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1395 tasks      | elapsed: 186.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1396 tasks      | elapsed: 186.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 1397 tasks      | elapsed: 187.1min
[Parallel(n_jobs=-1)]: Done 1398 tasks      | elapsed: 187.1min
Pickli

##### The best score and associated parameters followed by a table of more detailed results

In [12]:
imbdLogRegGSCV.best_score_

0.8609199999999999

In [13]:
for param_name in sorted(imbdLogRegParam.keys()):
    print("%s: %r" % (param_name, imbdLogRegGSCV.best_params_[param_name]))

clf__C: 1.0
clf__class_weight: 'balanced'
clf__dual: False
clf__fit_intercept: False
clf__intercept_scaling: 1.0
clf__l1_ratio: None
clf__max_iter: 1000
clf__multi_class: 'auto'
clf__penalty: 'l2'
clf__random_state: None
clf__solver: 'sag'
clf__tol: 0.001


In [14]:
df = pd.DataFrame(imbdLogRegGSCV.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__C,param_clf__class_weight,param_clf__dual,param_clf__fit_intercept,param_clf__intercept_scaling,param_clf__l1_ratio,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,29.714317,1.371448,0.977086,0.112440,0.1,None,False,False,1,None,...,0.8500,0.8720,0.8484,0.8388,0.8560,0.8736,0.8464,0.85376,0.010877,38
1,30.545114,2.065477,1.029880,0.191472,0.1,None,False,False,1,None,...,0.8500,0.8720,0.8484,0.8388,0.8560,0.8736,0.8464,0.85376,0.010877,38
2,29.726254,1.355728,0.962142,0.132071,0.1,None,False,False,1,None,...,0.8500,0.8720,0.8484,0.8388,0.8560,0.8736,0.8464,0.85376,0.010877,38
3,30.298656,1.289078,0.949410,0.164861,0.1,None,False,False,1,None,...,0.8500,0.8720,0.8484,0.8388,0.8560,0.8736,0.8464,0.85376,0.010877,38
4,30.957300,2.033507,1.028034,0.203939,0.1,None,False,False,1,None,...,0.8500,0.8720,0.8484,0.8388,0.8560,0.8736,0.8456,0.85368,0.010934,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,31.816608,2.141575,1.069161,0.239761,10,balanced,False,False,1,None,...,0.8248,0.8452,0.8364,0.8204,0.8436,0.8584,0.8384,0.83712,0.012964,116
140,31.142064,2.157731,1.087710,0.225734,10,balanced,False,False,1,None,...,0.8248,0.8452,0.8364,0.8208,0.8436,0.8584,0.8380,0.83712,0.012909,116
141,31.993284,2.105282,1.055546,0.215909,10,balanced,False,False,1,None,...,0.8248,0.8452,0.8364,0.8204,0.8436,0.8584,0.8384,0.83712,0.012964,116
142,31.337181,2.037590,1.035890,0.237439,10,balanced,False,False,1,None,...,0.8248,0.8452,0.8364,0.8204,0.8436,0.8588,0.8384,0.83712,0.012988,116


##### Use the pre-processing pipe to pre-process both the train and test data separately, then fit a model with the above best parameters to the train data and evaluate the result on the test data

In [49]:
imbdLogRegPreProcessingPipe.fit(imbd_train_data)
X_train_LogReg_imbd = imbdLogRegPreProcessingPipe.transform(imbd_train_data)
X_test_LogReg_imbd = imbdLogRegPreProcessingPipe.transform(imbd_test_data)
imbdLogRegDefault = LogisticRegression()
imbdLogRegDefault.fit(X_train_LogReg_imbd, imbd_train_target)
y_pred_LogReg_imbd_Default = imbdLogRegDefault.predict(X_test_LogReg_imbd)
metrics.accuracy_score(imbd_test_target, y_pred_LogReg_imbd_Default)

0.87872

In [51]:
imbdLogReg = LogisticRegression(C=1.0, class_weight='balanced', dual=False, fit_intercept=False, intercept_scaling=1.0, l1_ratio=None, max_iter=1000, multi_class='auto', penalty='l2', random_state=None, solver='sag', tol=0.001)
imbdLogReg.fit(X_train_LogReg_imbd, imbd_train_target)
y_pred_LogReg_imbd = imbdLogReg.predict(X_test_LogReg_imbd)
metrics.accuracy_score(imbd_test_target, y_pred_LogReg_imbd)

0.87892

In [52]:
metrics.confusion_matrix(imbd_test_target, y_pred_LogReg_imbd)

array([[11031,  1469],
       [ 1558, 10942]], dtype=int64)

In [53]:
print(metrics.classification_report(imbd_test_target, y_pred_LogReg_imbd))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88     12500
           1       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



# Decision Tree Classifier - Twenty News Group
##### Set up a pre-processing pipeline separately from the main-pipeline so that later we can pre-process before training our final model on the train dataset.  Define the parameters over which to do the GSCV and then perform the GSCV on the set of permutations of those parameters.

In [20]:
twentyDecTrePreProcessingPipe = Pipeline([
    ('vect', CountVectorizer(lowercase=True, max_df=0.4, max_features=10000, min_df=2, ngram_range=(1, 1), stop_words='english', strip_accents='ascii')),
    ('tfidf', TfidfTransformer()),
    ('scale', StandardScaler(with_mean=False)),
    ('norm', Normalizer())
])
twentyDecTrePipe = Pipeline([
    ('ppp', twentyDecTrePreProcessingPipe),
    ('clf', DecisionTreeClassifier())
])
twentyDecTreParam = {
    'clf__criterion': ['gini', 'entropy'],
    'clf__splitter': ['best'],
    'clf__max_depth': [None, 20, 100],
    'clf__min_samples_split': [0.01, 2, 10],
    'clf__min_samples_leaf': [0.01, 1, 10],
    'clf__min_weight_fraction_leaf': [0.0, 0.1],
    'clf__max_features': ['sqrt', 'log2', None],
    'clf__random_state': [None],
    'clf__max_leaf_nodes': [None, 100, 1000],
    'clf__min_impurity_decrease': [0.0, 0.1],
    'clf__class_weight': ['balanced', None],
    'clf__ccp_alpha': [0.0, 0.00001, 0.01]
}
twentyDecTreGSCV = RandomizedSearchCV(twentyDecTrePipe, twentyDecTreParam, n_iter=600, cv=10, n_jobs=-1, verbose=1000)
twentyDecTreGSCV = twentyDecTreGSCV.fit(twenty_train.data, twenty_train.target)

Fitting 10 folds for each of 600 candidates, totalling 6000 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.2s
Pickling array (shape=(11314,), dtype=int32).
Pick

[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed:   25.5s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   25.7s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:   27.3s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   27.6s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  43 tasks      | elapsed:   27.9s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Para

[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   51.2s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:   51.9s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  83 tasks      | elapsed:   52.0s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:   52.3s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  85 tasks      | elapsed:   53.6s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Para

[Parallel(n_jobs=-1)]: Done 123 tasks      | elapsed:  1.3min
Pickling array (shape=(11314,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  1.3min
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 125 tasks      | elapsed:  1.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 126 tasks      | elapsed:  1.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 127 tasks      | elapsed:  1.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Para

[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed:  1.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 167 tasks      | elapsed:  1.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  1.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 169 tasks      | elapsed:  1.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Para

...

[Parallel(n_jobs=-1)]: Done 5823 tasks      | elapsed: 64.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5824 tasks      | elapsed: 64.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5825 tasks      | elapsed: 64.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5826 tasks      | elapsed: 64.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5827 tasks      | elapsed: 64.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).


[Parallel(n_jobs=-1)]: Done 5864 tasks      | elapsed: 64.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5865 tasks      | elapsed: 64.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5866 tasks      | elapsed: 64.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5867 tasks      | elapsed: 64.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5868 tasks      | elapsed: 64.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).


[Parallel(n_jobs=-1)]: Done 5905 tasks      | elapsed: 65.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5906 tasks      | elapsed: 65.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5907 tasks      | elapsed: 65.1min
[Parallel(n_jobs=-1)]: Done 5908 tasks      | elapsed: 65.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5909 tasks      | elapsed: 65.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).


[Parallel(n_jobs=-1)]: Done 5946 tasks      | elapsed: 65.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5947 tasks      | elapsed: 65.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5948 tasks      | elapsed: 65.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5949 tasks      | elapsed: 65.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5950 tasks      | elapsed: 65.6min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).


[Parallel(n_jobs=-1)]: Done 5987 tasks      | elapsed: 65.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5988 tasks      | elapsed: 65.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5989 tasks      | elapsed: 65.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5990 tasks      | elapsed: 65.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 5991 tasks      | elapsed: 65.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).


##### The best score and associated parameters followed by a table of more detailed results

In [28]:
twentyDecTreGSCV.best_score_

0.48374042796487055

In [29]:
for param_name in sorted(twentyDecTreParam.keys()):
    print("%s: %r" % (param_name, twentyDecTreGSCV.best_params_[param_name]))

clf__ccp_alpha: 0.0
clf__class_weight: 'balanced'
clf__criterion: 'gini'
clf__max_depth: None
clf__max_features: None
clf__max_leaf_nodes: 1000
clf__min_impurity_decrease: 0.0
clf__min_samples_leaf: 1
clf__min_samples_split: 10
clf__min_weight_fraction_leaf: 0.0
clf__random_state: None
clf__splitter: 'best'


In [30]:
df = pd.DataFrame(twentyDecTreGSCV.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__splitter,param_clf__random_state,param_clf__min_weight_fraction_leaf,param_clf__min_samples_split,param_clf__min_samples_leaf,param_clf__min_impurity_decrease,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,2.159310,0.027392,0.203735,0.031580,best,None,0,0.01,1,0,...,0.053004,0.053050,0.053050,0.053050,0.053050,0.053050,0.053050,0.053032,0.000023,237
1,2.122121,0.031926,0.199736,0.023858,best,None,0,10,10,0.1,...,0.053004,0.053050,0.053050,0.053050,0.053050,0.053050,0.053050,0.053032,0.000023,237
2,2.207689,0.077093,0.213432,0.039060,best,None,0.1,10,0.01,0.1,...,0.050353,0.052166,0.047745,0.047745,0.047745,0.047745,0.051282,0.049584,0.001595,436
3,2.141015,0.093643,0.195437,0.031228,best,None,0,2,1,0.1,...,0.053004,0.053050,0.053050,0.053050,0.053050,0.053050,0.053050,0.053032,0.000023,237
4,2.103727,0.057623,0.184042,0.024366,best,None,0.1,2,0.01,0.1,...,0.050353,0.052166,0.047745,0.047745,0.047745,0.047745,0.051282,0.049584,0.001595,436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,1.954580,0.028542,0.171094,0.018606,best,None,0.1,0.01,10,0,...,0.053004,0.053050,0.053050,0.053050,0.053050,0.053050,0.053050,0.054003,0.001940,228
596,1.978296,0.034718,0.174104,0.018622,best,None,0.1,0.01,0.01,0.1,...,0.050353,0.052166,0.047745,0.047745,0.047745,0.047745,0.051282,0.049584,0.001595,436
597,2.025263,0.040121,0.164343,0.015963,best,None,0.1,10,1,0.1,...,0.050353,0.052166,0.047745,0.047745,0.047745,0.047745,0.051282,0.049584,0.001595,436
598,2.048540,0.052571,0.166916,0.019398,best,None,0,2,1,0.1,...,0.050353,0.052166,0.047745,0.047745,0.047745,0.047745,0.051282,0.049584,0.001595,436


##### Use the pre-processing pipe to pre-process both the train and test data separately, then fit a model with the above best parameters to the train data and evaluate the result on the test data

In [54]:
twentyDecTrePreProcessingPipe.fit(twenty_train.data)
X_train_DecTre_twenty = twentyDecTrePreProcessingPipe.transform(twenty_train.data)
X_test_DecTre_twenty = twentyDecTrePreProcessingPipe.transform(twenty_test.data)
twentyDecTreDefault = DecisionTreeClassifier()
twentyDecTreDefault.fit(X_train_DecTre_twenty, twenty_train.target)
y_pred_DecTre_twenty_Default = twentyDecTreDefault.predict(X_test_DecTre_twenty)
metrics.accuracy_score(twenty_test.target, y_pred_DecTre_twenty_Default)

0.4368029739776952

In [55]:
twentyDecTre = DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features=None, max_leaf_nodes=1000, min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=10, min_weight_fraction_leaf=0.0, random_state=None, splitter='best')
twentyDecTre.fit(X_train_DecTre_twenty, twenty_train.target)
y_pred_DecTre_twenty = twentyDecTre.predict(X_test_DecTre_twenty)
metrics.accuracy_score(twenty_test.target, y_pred_DecTre_twenty)

0.43866171003717475

In [56]:
metrics.confusion_matrix(twenty_test.target, y_pred_DecTre_twenty)

array([[ 76,  11,   0,   0,   0,   2,   4,  76,   2,   3,   4,   1,   2,
          8,   6,  51,  11,   6,  22,  34],
       [  4, 186,  27,  13,  11,  29,  13,  50,   0,   5,   2,   4,  22,
          2,  10,   0,   1,   3,   7,   0],
       [  0,  36, 163,  45,  17,  28,   3,  58,   2,   2,   1,   1,   6,
          4,   8,   0,   5,   2,   9,   4],
       [  0,  34,  27, 157,  35,  17,  22,  33,   1,   0,   1,   3,  40,
          3,   3,   1,   5,   0,   8,   2],
       [  1,  41,   7,  37, 151,   6,  20,  68,   0,   1,   1,   2,  21,
          8,   5,   1,   2,   2,  10,   1],
       [  1,  58,  43,  10,   7, 164,   4,  51,   2,   0,   0,   8,  24,
          2,   3,   1,   3,   2,   8,   4],
       [  1,  23,   7,  22,  21,   6, 211,  56,   6,   1,   6,   3,  13,
          3,   2,   2,   2,   0,   4,   1],
       [  2,  11,   6,   3,   5,   4,  19, 245,  23,   5,   4,   2,  18,
          5,   6,   0,   7,   8,  16,   7],
       [  7,   5,   2,   3,   8,   3,  12,  97, 201,   5,   1,  

In [57]:
print(metrics.classification_report(twenty_test.target, y_pred_DecTre_twenty, target_names=twenty_test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.36      0.24      0.29       319
           comp.graphics       0.35      0.48      0.41       389
 comp.os.ms-windows.misc       0.53      0.41      0.46       394
comp.sys.ibm.pc.hardware       0.46      0.40      0.43       392
   comp.sys.mac.hardware       0.54      0.39      0.45       385
          comp.windows.x       0.57      0.42      0.48       395
            misc.forsale       0.56      0.54      0.55       390
               rec.autos       0.16      0.62      0.26       396
         rec.motorcycles       0.78      0.51      0.61       398
      rec.sport.baseball       0.69      0.49      0.57       397
        rec.sport.hockey       0.68      0.62      0.65       399
               sci.crypt       0.75      0.44      0.55       396
         sci.electronics       0.33      0.33      0.33       393
                 sci.med       0.62      0.40      0.49       396
         

# Decision Tree Classifier - IBMD Movie Reviews
##### Set up a pre-processing pipeline separately from the main-pipeline so that later we can pre-process before training our final model on the train dataset.  Define the parameters over which to do the GSCV and then perform the GSCV on the set of permutations of those parameters.

In [32]:
imbdDecTrePreProcessingPipe = Pipeline([
    ('vect', CountVectorizer(lowercase=True, max_df=0.4, max_features=10000, min_df=3, ngram_range=(1, 3), stop_words='english', strip_accents='ascii')),
    ('tfidf', TfidfTransformer()),
    ('scale', StandardScaler(with_mean=False)),
    ('norm', Normalizer())
])
imbdDecTrePipe = Pipeline([
    ('ppp', imbdDecTrePreProcessingPipe),
    ('clf', DecisionTreeClassifier())
])
imbdDecTreParam = {
    'clf__criterion': ['gini', 'entropy'],
    'clf__splitter': ['best'],
    'clf__max_depth': [None, 20, 100],
    'clf__min_samples_split': [0.01, 2, 10],
    'clf__min_samples_leaf': [0.01, 1, 10],
    'clf__min_weight_fraction_leaf': [0.0, 0.1],
    'clf__max_features': ['sqrt', 'log2', None],
    'clf__random_state': [None],
    'clf__max_leaf_nodes': [None, 100, 1000],
    'clf__min_impurity_decrease': [0.0, 0.1],
    'clf__class_weight': ['balanced', None],
    'clf__ccp_alpha': [0.0, 0.00001, 0.01]
}
imbdDecTreGSCV = RandomizedSearchCV(imbdDecTrePipe, imbdDecTreParam, n_iter=100, cv=10, n_jobs=-1, verbose=1000)
imbdDecTreGSCV = imbdDecTreGSCV.fit(imbd_train_data, imbd_train_target)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   32.7s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   33.1s
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   33.8s
Pickling array (shape=(22500,), dtype=int32).
Pickling array (sha

[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:  6.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:  7.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  55 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  7.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)

[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 106 tasks      | elapsed: 13.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 107 tasks      | elapsed: 13.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed: 13.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 14.4min
[Parallel(n_jobs=-1)]: Done 110 tasks      | elapsed: 14.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 111 tasks      | elapsed: 14.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 161 tasks      | elapsed: 21.0min
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed: 21.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 163 tasks      | elapsed: 21.0min
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed: 21.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 21.5min
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed: 21.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 214 tasks      | elapsed: 27.7min
[Parallel(n_jobs=-1)]: Done 215 tasks      | elapsed: 27.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed: 27.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed: 28.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 218 tasks      | elapsed: 28.2min
[Parallel(n_jobs=-1)]: Done 219 tasks      | elapsed: 28.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 220 tasks      | elapsed: 28.2min
Pickling array (shap

...

[Parallel(n_jobs=-1)]: Done 753 tasks      | elapsed: 98.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 754 tasks      | elapsed: 98.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 755 tasks      | elapsed: 98.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 756 tasks      | elapsed: 98.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 757 tasks      | elapsed: 98.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 758 tasks      | elapsed: 98.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 759 tasks      | elapsed: 98.8min
Pickling array (shap

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 807 tasks      | elapsed: 105.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 808 tasks      | elapsed: 105.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 809 tasks      | elapsed: 105.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 810 tasks      | elapsed: 105.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 811 tasks      | elapsed: 105.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 812 tasks      | elapsed: 105.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,),

[Parallel(n_jobs=-1)]: Done 860 tasks      | elapsed: 112.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 861 tasks      | elapsed: 112.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 862 tasks      | elapsed: 112.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 863 tasks      | elapsed: 112.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 864 tasks      | elapsed: 112.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 865 tasks      | elapsed: 112.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed: 112.9min
Pickling arra

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 914 tasks      | elapsed: 119.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 915 tasks      | elapsed: 119.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 916 tasks      | elapsed: 119.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 917 tasks      | elapsed: 119.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 918 tasks      | elapsed: 119.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 919 tasks      | elapsed: 119.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,),

[Parallel(n_jobs=-1)]: Done 967 tasks      | elapsed: 126.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 968 tasks      | elapsed: 126.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 969 tasks      | elapsed: 126.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 970 tasks      | elapsed: 126.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 971 tasks      | elapsed: 126.8min
[Parallel(n_jobs=-1)]: Done 972 tasks      | elapsed: 126.8min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 973 tasks      | elapsed: 127.0min
Pickling arra

##### The best score and associated parameters followed by a table of more detailed results

In [33]:
imbdDecTreGSCV.best_score_

0.7581999999999999

In [34]:
for param_name in sorted(imbdDecTreParam.keys()):
    print("%s: %r" % (param_name, imbdDecTreGSCV.best_params_[param_name]))

clf__ccp_alpha: 0.0
clf__class_weight: 'balanced'
clf__criterion: 'gini'
clf__max_depth: None
clf__max_features: None
clf__max_leaf_nodes: 100
clf__min_impurity_decrease: 0.0
clf__min_samples_leaf: 10
clf__min_samples_split: 0.01
clf__min_weight_fraction_leaf: 0.0
clf__random_state: None
clf__splitter: 'best'


In [35]:
df = pd.DataFrame(imbdDecTreGSCV.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__splitter,param_clf__random_state,param_clf__min_weight_fraction_leaf,param_clf__min_samples_split,param_clf__min_samples_leaf,param_clf__min_impurity_decrease,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,30.071768,0.696179,0.972715,0.051275,best,None,0,10,1,0,...,0.6220,0.5780,0.5788,0.5548,0.5832,0.5864,0.5832,0.57256,0.023578,23
1,28.938505,0.210500,0.982921,0.049406,best,None,0,0.01,1,0.1,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50000,0.000000,36
2,29.176671,0.224079,0.972861,0.077675,best,None,0,10,0.01,0.1,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50000,0.000000,36
3,28.909525,0.123085,0.969456,0.039156,best,None,0,0.01,1,0.1,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50000,0.000000,36
4,29.077752,0.210260,0.984362,0.104584,best,None,0,0.01,10,0.1,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50000,0.000000,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,33.116860,0.294575,0.868960,0.037897,best,None,0,2,10,0,...,0.6948,0.6828,0.6732,0.6956,0.6988,0.6608,0.6848,0.68384,0.011557,6
96,29.362136,0.206761,0.897684,0.051543,best,None,0.1,2,1,0,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50000,0.000000,36
97,29.804362,0.219347,0.883946,0.038015,best,None,0.1,2,0.01,0,...,0.6216,0.6164,0.6072,0.6080,0.6172,0.6104,0.6184,0.61300,0.005073,19
98,29.425650,0.132061,0.923862,0.045093,best,None,0,0.01,0.01,0,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5336,0.51504,0.024799,28


##### Use the pre-processing pipe to pre-process both the train and test data separately, then fit a model with the above best parameters to the train data and evaluate the result on the test data

In [58]:
imbdDecTrePreProcessingPipe.fit(imbd_train_data)
X_train_DecTre_imbd = imbdDecTrePreProcessingPipe.transform(imbd_train_data)
X_test_DecTre_imbd = imbdDecTrePreProcessingPipe.transform(imbd_test_data)
imbdDecTreDefault = DecisionTreeClassifier()
imbdDecTreDefault.fit(X_train_DecTre_imbd, imbd_train_target)
y_pred_DecTre_imbd_Default = imbdDecTreDefault.predict(X_test_DecTre_imbd)
metrics.accuracy_score(imbd_test_target, y_pred_DecTre_imbd_Default)

0.70788

In [59]:
imbdDecTre = DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini', max_depth=None, max_features=None, max_leaf_nodes=100, min_impurity_decrease=0.0, min_samples_leaf=10, min_samples_split=0.01, min_weight_fraction_leaf=0.0, random_state=None, splitter='best')
imbdDecTre.fit(X_train_DecTre_imbd, imbd_train_target)
y_pred_DecTre_imbd = imbdDecTre.predict(X_test_DecTre_imbd)
metrics.accuracy_score(imbd_test_target, y_pred_DecTre_imbd)

0.762

In [60]:
metrics.confusion_matrix(imbd_test_target, y_pred_DecTre_imbd)

array([[9443, 3057],
       [2893, 9607]], dtype=int64)

In [61]:
print(metrics.classification_report(imbd_test_target, y_pred_DecTre_imbd))

              precision    recall  f1-score   support

           0       0.77      0.76      0.76     12500
           1       0.76      0.77      0.76     12500

    accuracy                           0.76     25000
   macro avg       0.76      0.76      0.76     25000
weighted avg       0.76      0.76      0.76     25000



# Linear SVC - Twenty News Group
##### Set up a pre-processing pipeline separately from the main-pipeline so that later we can pre-process before training our final model on the train dataset.  Define the parameters over which to do the GSCV and then perform the GSCV on the set of permutations of those parameters.

In [5]:
twentyLinSVCPreProcessingPipe = Pipeline([
    ('vect', CountVectorizer(lowercase=True, max_df=0.3, max_features=100000, min_df=2, ngram_range=(1, 2), stop_words='english', strip_accents='ascii')),
    ('tfidf', TfidfTransformer()),
    ('scale', StandardScaler(with_mean=False)),
    ('norm', Normalizer())
])
twentyLinSVCPipe = Pipeline([
    ('ppp', twentyLinSVCPreProcessingPipe),
    ('clf', LinearSVC())
])
twentyLinSVCParam = {
    'clf__penalty': ['l2'],
    'clf__loss': ['squared_hinge', 'hinge'],
    'clf__dual': [True, False],
    'clf__tol': [0.00001, 0.0001, 0.001],
    'clf__C': [0.1, 1.0, 2.5, 10.0],
    'clf__multi_class': ['ovr'],
    'clf__fit_intercept': [True, False],
    'clf__intercept_scaling': [0.1, 1.0, 2.0, 10.0],
    'clf__random_state': [None],
    'clf__max_iter': [100, 1000, 10000]
}
twentyLinSVCGSCV = RandomizedSearchCV(twentyLinSVCPipe, twentyLinSVCParam, n_iter=100, cv=10, n_jobs=-1, verbose=1000)
twentyLinSVCGSCV = twentyLinSVCGSCV.fit(twenty_train.data, twenty_train.target)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   39.7s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shap

Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:  5.6min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:  5.6min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.6min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  43 tasks      | elapsed:  5.9min
[Para

[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:  8.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  8.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  83 tasks      | elapsed:  9.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:  9.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Para

[Parallel(n_jobs=-1)]: Done 122 tasks      | elapsed: 15.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 123 tasks      | elapsed: 15.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 15.8min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 125 tasks      | elapsed: 15.8min
[Parallel(n_jobs=-1)]: Done 126 tasks      | elapsed: 15.8min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickl

[Parallel(n_jobs=-1)]: Done 163 tasks      | elapsed: 20.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed: 20.5min

Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 20.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed: 21.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 167 tasks      | elapsed: 21.2min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed: 21.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dt

...

Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 826 tasks      | elapsed: 150.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 827 tasks      | elapsed: 150.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed: 150.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 829 tasks      | elapsed: 150.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 830 tasks      | elapsed: 151.1min


Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 867 tasks      | elapsed: 157.4min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 868 tasks      | elapsed: 157.4min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 869 tasks      | elapsed: 157.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 870 tasks      | elapsed: 158.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 871 tasks      | elapsed: 158.1min


Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 908 tasks      | elapsed: 161.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 909 tasks      | elapsed: 161.6min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 910 tasks      | elapsed: 161.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 911 tasks      | elapsed: 161.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 912 tasks      | elapsed: 161.8min


Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 949 tasks      | elapsed: 168.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 950 tasks      | elapsed: 168.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 951 tasks      | elapsed: 168.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 952 tasks      | elapsed: 168.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 953 tasks      | elapsed: 168.4min


Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 990 tasks      | elapsed: 180.8min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 991 tasks      | elapsed: 180.8min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 992 tasks      | elapsed: 180.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 993 tasks      | elapsed: 181.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (s

##### The best score and associated parameters followed by a table of more detailed results

In [6]:
twentyLinSVCGSCV.best_score_

0.735993039087958

In [7]:
for param_name in sorted(twentyLinSVCParam.keys()):
    print("%s: %r" % (param_name, twentyLinSVCGSCV.best_params_[param_name]))

clf__C: 0.1
clf__dual: False
clf__fit_intercept: False
clf__intercept_scaling: 10.0
clf__loss: 'squared_hinge'
clf__max_iter: 100
clf__multi_class: 'ovr'
clf__penalty: 'l2'
clf__random_state: None
clf__tol: 0.001


In [8]:
df = pd.DataFrame(twentyLinSVCGSCV.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__tol,param_clf__random_state,param_clf__penalty,param_clf__multi_class,param_clf__max_iter,param_clf__loss,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,33.864849,2.323784,0.939124,0.159812,0.001,None,l2,ovr,1000,squared_hinge,...,0.727032,0.735632,0.717065,0.723254,0.705570,0.740053,0.709107,0.720877,0.011041,51
1,35.559492,2.223074,0.830871,0.112243,1e-05,None,l2,ovr,10000,squared_hinge,...,0.727032,0.735632,0.717065,0.723254,0.705570,0.740053,0.709107,0.720877,0.011041,51
2,17.884783,1.032885,0.754582,0.100721,0.001,None,l2,ovr,100,squared_hinge,...,0.741166,0.754200,0.716180,0.740053,0.732095,0.755084,0.720601,0.732457,0.013948,13
3,36.022897,1.202444,0.822385,0.109832,0.001,None,l2,ovr,10000,squared_hinge,...,0.722615,0.732980,0.716180,0.714412,0.709107,0.733864,0.702918,0.716812,0.010059,65
4,20.145964,0.483588,0.845820,0.081364,0.001,None,l2,ovr,10000,squared_hinge,...,0.740283,0.754200,0.716180,0.741821,0.729443,0.755084,0.720601,0.732457,0.013905,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,26.071607,0.953454,0.771037,0.101742,1e-05,None,l2,ovr,1000,squared_hinge,...,0.727032,0.735632,0.717065,0.723254,0.705570,0.740053,0.709107,0.720877,0.011041,51
96,197.830945,20.053002,0.787909,0.079287,1e-05,None,l2,ovr,1000,hinge,...,0.730565,0.742706,0.715296,0.730327,0.717949,0.741821,0.719717,0.725916,0.010075,28
97,22.292704,0.678598,0.841203,0.072322,1e-05,None,l2,ovr,10000,squared_hinge,...,0.741166,0.762157,0.724138,0.746242,0.730327,0.755084,0.723254,0.735993,0.013652,1
98,53.874695,2.848559,0.839008,0.106896,1e-05,None,l2,ovr,10000,squared_hinge,...,0.722615,0.732980,0.716180,0.714412,0.709107,0.733864,0.702918,0.716812,0.010059,65


##### Use the pre-processing pipe to pre-process both the train and test data separately, then fit a model with the above best parameters to the train data and evaluate the result on the test data

In [13]:
twentyLinSVCPreProcessingPipe.fit(twenty_train.data)
X_train_LinSVC_twenty = twentyLinSVCPreProcessingPipe.transform(twenty_train.data)
X_test_LinSVC_twenty = twentyLinSVCPreProcessingPipe.transform(twenty_test.data)
twentyLinSVCDefault = LinearSVC()
twentyLinSVCDefault.fit(X_train_LinSVC_twenty, twenty_train.target)
y_pred_LinSVC_twenty_Default = twentyLinSVCDefault.predict(X_test_LinSVC_twenty)
metrics.accuracy_score(twenty_test.target, y_pred_LinSVC_twenty_Default)

0.6540095592140202

In [14]:
twentyLinSVC = LinearSVC(C=0.1, dual=False, fit_intercept=False, intercept_scaling=10.0, loss='squared_hinge', max_iter=100, multi_class='ovr', penalty='l2', random_state=None, tol=0.001)
twentyLinSVC.fit(X_train_LinSVC_twenty, twenty_train.target)
y_pred_LinSVC_twenty = twentyLinSVC.predict(X_test_LinSVC_twenty)
metrics.accuracy_score(twenty_test.target, y_pred_LinSVC_twenty)

0.6692777482740307

In [15]:
metrics.confusion_matrix(twenty_test.target, y_pred_LinSVC_twenty)

array([[136,   0,   2,   1,   2,   1,   3,   1,   4,   2,   9,   5,   2,
          6,  10,  74,  16,  19,  10,  16],
       [ 10, 266,  13,  17,   7,  23,   9,   3,   4,   2,   0,  17,   6,
          0,   7,   1,   0,   2,   0,   2],
       [ 18,  21, 224,  35,  11,  28,   6,   1,   4,   0,   2,   7,   5,
          3,   9,   6,   4,   1,   6,   3],
       [  7,  21,  33, 247,  27,   6,  14,   3,   1,   0,   1,   3,  17,
          2,   6,   0,   1,   2,   0,   1],
       [ 17,   7,  16,  28, 249,   6,  16,   4,   1,   0,   1,   4,  13,
          7,   7,   2,   4,   1,   1,   1],
       [  7,  43,  30,   6,   0, 280,   4,   0,   2,   0,   1,   8,   3,
          3,   5,   2,   0,   0,   0,   1],
       [  9,   2,   5,  23,  10,   1, 305,   7,   5,   3,   2,   1,   4,
          2,   3,   3,   1,   1,   2,   1],
       [ 26,   2,   2,   1,   8,   3,  15, 273,  17,   2,   2,   4,  14,
          5,   2,   5,   4,   3,   7,   1],
       [ 18,   3,   6,   3,   1,   3,   6,  26, 259,   6,   4,  

In [16]:
print(metrics.classification_report(twenty_test.target, y_pred_LinSVC_twenty, target_names=twenty_test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.29      0.43      0.34       319
           comp.graphics       0.64      0.68      0.66       389
 comp.os.ms-windows.misc       0.62      0.57      0.59       394
comp.sys.ibm.pc.hardware       0.63      0.63      0.63       392
   comp.sys.mac.hardware       0.74      0.65      0.69       385
          comp.windows.x       0.76      0.71      0.73       395
            misc.forsale       0.71      0.78      0.74       390
               rec.autos       0.74      0.69      0.71       396
         rec.motorcycles       0.76      0.65      0.70       398
      rec.sport.baseball       0.89      0.80      0.84       397
        rec.sport.hockey       0.86      0.90      0.88       399
               sci.crypt       0.71      0.73      0.72       396
         sci.electronics       0.67      0.52      0.59       393
                 sci.med       0.74      0.76      0.75       396
         

# Linear SVC - IBMD Movie Reviews
##### Set up a pre-processing pipeline separately from the main-pipeline so that later we can pre-process before training our final model on the train dataset.  Define the parameters over which to do the GSCV and then perform the GSCV on the set of permutations of those parameters.

In [9]:
imbdLinSVCPreProcessingPipe = Pipeline([
    ('vect', CountVectorizer(lowercase=True, max_df=0.5, max_features=10000, min_df=3, ngram_range=(1, 3), stop_words='english', strip_accents='ascii')),
    ('tfidf', TfidfTransformer()),
    ('scale', StandardScaler(with_mean=False)),
    ('norm', Normalizer())
])
imbdLinSVCPipe = Pipeline([
    ('ppp', imbdLinSVCPreProcessingPipe),
    ('clf', LinearSVC())
])
imbdLinSVCParam = {
    'clf__penalty': ['l2'],
    'clf__loss': ['squared_hinge', 'hinge'],
    'clf__dual': [True, False],
    'clf__tol': [0.00001, 0.0001, 0.001],
    'clf__C': [0.1, 1.0, 2.5, 10.0],
    'clf__multi_class': ['ovr'],
    'clf__fit_intercept': [True, False],
    'clf__intercept_scaling': [0.1, 1.0, 2.0, 10.0],
    'clf__random_state': [None],
    'clf__max_iter': [100, 1000, 10000]
}
imbdLinSVCGSCV = RandomizedSearchCV(imbdLinSVCPipe, imbdLinSVCParam, n_iter=100, cv=10, n_jobs=-1, verbose=1000)
imbdLinSVCGSCV = imbdLinSVCGSCV.fit(imbd_train_data, imbd_train_target)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (sha

[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed: 13.0min
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed: 13.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 13.9min
Pickling array (shape=(22500,), dtype=int32).[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed: 13.9min

Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  55 tasks      | elapsed: 14.0min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed: 14.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 14.9min
Pickling array (shap

[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 26.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 106 tasks      | elapsed: 26.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 107 tasks      | elapsed: 26.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed: 26.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done 110 tasks      | elapsed: 27.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 111 tasks      | elapsed: 27.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shap

[Parallel(n_jobs=-1)]: Done 159 tasks      | elapsed: 39.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed: 39.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 161 tasks      | elapsed: 39.9min
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed: 39.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 163 tasks      | elapsed: 40.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed: 40.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 40.9min
[Parallel(n_jobs=-1)

[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 52.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 214 tasks      | elapsed: 52.8min
[Parallel(n_jobs=-1)]: Done 215 tasks      | elapsed: 52.8min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed: 52.8min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed: 53.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 218 tasks      | elapsed: 53.7min
[Parallel(n_jobs=-1)]: Done 219 tasks      | elapsed: 53.8min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shap

...

[Parallel(n_jobs=-1)]: Done 750 tasks      | elapsed: 184.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 751 tasks      | elapsed: 185.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 752 tasks      | elapsed: 185.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 753 tasks      | elapsed: 185.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 754 tasks      | elapsed: 185.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 755 tasks      | elapsed: 186.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 756 tasks      | elapsed: 186.2min
Pickling arra

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 804 tasks      | elapsed: 198.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 805 tasks      | elapsed: 198.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 806 tasks      | elapsed: 198.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 807 tasks      | elapsed: 199.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 808 tasks      | elapsed: 199.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 809 tasks      | elapsed: 199.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,),

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 858 tasks      | elapsed: 211.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 859 tasks      | elapsed: 211.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 860 tasks      | elapsed: 211.8min
[Parallel(n_jobs=-1)]: Done 861 tasks      | elapsed: 211.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 862 tasks      | elapsed: 211.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 863 tasks      | ela

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 911 tasks      | elapsed: 224.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 912 tasks      | elapsed: 224.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 913 tasks      | elapsed: 224.9min
[Parallel(n_jobs=-1)]: Done 914 tasks      | elapsed: 224.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 915 tasks      | elapsed: 225.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 916 tasks      | elapsed: 225.8min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,),

[Parallel(n_jobs=-1)]: Done 964 tasks      | elapsed: 237.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 965 tasks      | elapsed: 237.4min
[Parallel(n_jobs=-1)]: Done 966 tasks      | elapsed: 237.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 967 tasks      | elapsed: 237.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 968 tasks      | elapsed: 238.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 969 tasks      | elapsed: 238.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 970 tasks      | elapsed: 238.3min
Pickling arra

##### The best score and associated parameters followed by a table of more detailed results

In [10]:
imbdLinSVCGSCV.best_score_

0.8595600000000001

In [11]:
for param_name in sorted(imbdLinSVCParam.keys()):
    print("%s: %r" % (param_name, imbdLinSVCGSCV.best_params_[param_name]))

clf__C: 0.1
clf__dual: True
clf__fit_intercept: False
clf__intercept_scaling: 1.0
clf__loss: 'squared_hinge'
clf__max_iter: 1000
clf__multi_class: 'ovr'
clf__penalty: 'l2'
clf__random_state: None
clf__tol: 0.001


In [12]:
df = pd.DataFrame(imbdLinSVCGSCV.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__tol,param_clf__random_state,param_clf__penalty,param_clf__multi_class,param_clf__max_iter,param_clf__loss,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,62.250590,3.651770,0.000000,0.000000,0.0001,None,l2,ovr,10000,hinge,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80
1,56.134810,0.552501,0.000000,0.000000,0.0001,None,l2,ovr,1000,hinge,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82
2,55.936576,0.856541,2.462786,0.171268,0.0001,None,l2,ovr,1000,squared_hinge,...,0.8504,0.8716,0.8556,0.8468,0.8648,0.8788,0.8616,0.85920,0.011559,12
3,55.836285,0.414416,2.394331,0.148893,0.0001,None,l2,ovr,1000,squared_hinge,...,0.8188,0.8376,0.8312,0.8172,0.8408,0.8448,0.8340,0.82984,0.012594,31
4,56.305658,1.523448,2.293344,0.155701,0.0001,None,l2,ovr,10000,squared_hinge,...,0.8188,0.8376,0.8328,0.8164,0.8392,0.8440,0.8340,0.82960,0.012910,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,54.589769,0.411892,2.183628,0.086200,0.001,None,l2,ovr,10000,squared_hinge,...,0.7896,0.8128,0.7976,0.7892,0.8028,0.8172,0.8044,0.79980,0.011911,68
96,54.344985,0.483954,0.000000,0.000000,0.001,None,l2,ovr,100,hinge,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76
97,55.557296,1.734621,2.266834,0.182125,0.0001,None,l2,ovr,1000,squared_hinge,...,0.8064,0.8224,0.8152,0.8084,0.8212,0.8336,0.8240,0.81628,0.011123,46
98,58.431151,1.867144,2.368122,0.347066,0.0001,None,l2,ovr,100,squared_hinge,...,0.8504,0.8716,0.8560,0.8464,0.8648,0.8788,0.8612,0.85928,0.011344,9


##### Use the pre-processing pipe to pre-process both the train and test data separately, then fit a model with the above best parameters to the train data and evaluate the result on the test data

In [17]:
imbdLinSVCPreProcessingPipe.fit(imbd_train_data)
X_train_LinSVC_imbd = imbdLinSVCPreProcessingPipe.transform(imbd_train_data)
X_test_LinSVC_imbd = imbdLinSVCPreProcessingPipe.transform(imbd_test_data)
imbdLinSVCDefault = LinearSVC()
imbdLinSVCDefault.fit(X_train_LinSVC_imbd, imbd_train_target)
y_pred_LinSVC_imbd_Default = imbdLinSVCDefault.predict(X_test_LinSVC_imbd)
metrics.accuracy_score(imbd_test_target, y_pred_LinSVC_imbd_Default)

0.85628

In [18]:
imbdLinSVC = LinearSVC(C=0.1, dual=True, fit_intercept=False, intercept_scaling=1.0, loss='squared_hinge', max_iter=1000, multi_class='ovr', penalty='l2', random_state=None, tol=0.001)
imbdLinSVC.fit(X_train_LinSVC_imbd, imbd_train_target)
y_pred_LinSVC_imbd = imbdLinSVC.predict(X_test_LinSVC_imbd)
metrics.accuracy_score(imbd_test_target, y_pred_LinSVC_imbd)

0.87712

In [19]:
metrics.confusion_matrix(imbd_test_target, y_pred_LinSVC_imbd)

array([[11000,  1500],
       [ 1572, 10928]], dtype=int64)

In [20]:
print(metrics.classification_report(imbd_test_target, y_pred_LinSVC_imbd))

              precision    recall  f1-score   support

           0       0.87      0.88      0.88     12500
           1       0.88      0.87      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



# Random Forest Classifier - Twenty News Group
##### Set up a pre-processing pipeline separately from the main-pipeline so that later we can pre-process before training our final model on the train dataset.  Define the parameters over which to do the GSCV and then perform the GSCV on the set of permutations of those parameters.

In [37]:
twentyRndForPreProcessingPipe = Pipeline([
    ('vect', CountVectorizer(lowercase=True, max_df=0.5, max_features=100000, min_df=2, ngram_range=(1, 2), stop_words='english', strip_accents='ascii')),
    ('tfidf', TfidfTransformer()),
    ('scale', StandardScaler(with_mean=False)),
    ('norm', Normalizer())
])
twentyRndForPipe = Pipeline([
    ('ppp', twentyRndForPreProcessingPipe),
    ('clf', RandomForestClassifier())
])
twentyRndForParam = {
    'clf__n_estimators': [10, 50, 100],
    'clf__criterion': ['gini', 'entropy'],
    'clf__max_depth': [None, 20, 100],
    'clf__min_samples_split': [0.01, 2, 10],
    'clf__min_samples_leaf': [0.01, 1, 10],
    'clf__min_weight_fraction_leaf': [0.0, 0.1],
    'clf__max_features': ['sqrt', 'log2', None],
    'clf__max_leaf_nodes': [None, 100, 1000],
    'clf__min_impurity_decrease': [0.0, 0.1],
    'clf__bootstrap': [True],
    'clf__oob_score': [True, False],
    'clf__random_state': [None],
    'clf__class_weight': ['balanced', None],
    'clf__ccp_alpha': [0.0, 0.00001, 0.01],
    'clf__max_samples': [0.1, 0.5, None]
}
twentyRndForGSCV = RandomizedSearchCV(twentyRndForPipe, twentyRndForParam, n_iter=100, cv=10, n_jobs=-1, verbose=1000)
twentyRndForGSCV = twentyRndForGSCV.fit(twenty_train.data, twenty_train.target)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    7.6s
Pickling array (shape=(11314,), dtype=int32).
Pick

Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed:  1.4min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:  1.4min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:  1.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  43 tasks      | elapsed:  1.5min
Pickl

Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  2.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  83 tasks      | elapsed:  2.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  84 tasks      | elapsed:  2.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  85 tasks      | elapsed:  3.0min
Pickl

Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 123 tasks      | elapsed:  4.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:  4.7min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 125 tasks      | elapsed:  4.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 126 tasks      | elapsed:  4.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape

[Parallel(n_jobs=-1)]: Done 163 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:  6.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  6.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed:  6.2min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 167 tasks      | elapsed:  6.3min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Para

...

Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 827 tasks      | elapsed: 42.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 828 tasks      | elapsed: 42.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 829 tasks      | elapsed: 42.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 830 tasks      | elapsed: 42.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 831 tasks      | elapsed: 42.1min
Pickl

[Parallel(n_jobs=-1)]: Done 868 tasks      | elapsed: 43.4min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 869 tasks      | elapsed: 43.4min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 870 tasks      | elapsed: 43.4min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 871 tasks      | elapsed: 43.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 872 tasks      | elapsed: 43.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Para

Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10182,), dtype=int32).
Pickling array (shape=(1132,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 910 tasks      | elapsed: 46.9min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 911 tasks      | elapsed: 47.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 912 tasks      | elapsed: 47.0min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 913 tasks      | elapsed: 47.1min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 914 tasks      | elapsed: 47.1min
Pickl

[Parallel(n_jobs=-1)]: Done 951 tasks      | elapsed: 48.4min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 952 tasks      | elapsed: 48.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 953 tasks      | elapsed: 48.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 954 tasks      | elapsed: 48.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 955 tasks      | elapsed: 48.5min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Para

Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 993 tasks      | elapsed: 49.8min
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 995 out of 1000 | elapsed: 49.8min remaining:   14.9s
Pickling array (shape=(11314,), dtype=int32).
Pickling array (shape=(10183,), dtype=int32).
Pickling array (shape=(1131,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 997 out of 1000 | elapsed: 49.9min remaining:    8.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 50.0min finished


##### The best score and associated parameters followed by a table of more detailed results

In [38]:
twentyRndForGSCV.best_score_

0.6469003164903007

In [39]:
for param_name in sorted(twentyRndForParam.keys()):
    print("%s: %r" % (param_name, twentyRndForGSCV.best_params_[param_name]))

clf__bootstrap: True
clf__ccp_alpha: 1e-05
clf__class_weight: 'balanced'
clf__criterion: 'gini'
clf__max_depth: 100
clf__max_features: 'sqrt'
clf__max_leaf_nodes: None
clf__max_samples: 0.5
clf__min_impurity_decrease: 0.0
clf__min_samples_leaf: 1
clf__min_samples_split: 2
clf__min_weight_fraction_leaf: 0.0
clf__n_estimators: 100
clf__oob_score: False
clf__random_state: None


In [40]:
df = pd.DataFrame(twentyRndForGSCV.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__random_state,param_clf__oob_score,param_clf__n_estimators,param_clf__min_weight_fraction_leaf,param_clf__min_samples_split,param_clf__min_samples_leaf,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,7.007150,0.091731,0.378083,0.035510,None,False,100,0,0.01,10,...,0.052120,0.053050,0.053050,0.052166,0.053050,0.052166,0.052166,0.051353,0.003597,77
1,7.035602,0.118657,0.367221,0.043412,None,True,10,0.1,2,0.01,...,0.052120,0.053050,0.052166,0.053050,0.053050,0.053050,0.052166,0.052590,0.000600,67
2,7.773668,0.232084,0.448071,0.061901,None,False,100,0,2,1,...,0.619258,0.656941,0.602122,0.630416,0.641910,0.654288,0.607427,0.629134,0.018299,2
3,8.343767,0.287124,0.372383,0.039162,None,True,100,0.1,10,10,...,0.052120,0.053050,0.047745,0.041556,0.044209,0.052166,0.052166,0.050026,0.003887,84
4,9.103497,0.159488,0.320432,0.037430,None,True,50,0.1,10,1,...,0.063604,0.068081,0.068081,0.060124,0.066313,0.074271,0.065429,0.064788,0.004385,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6.791288,0.140614,0.303486,0.026505,None,True,10,0.1,10,1,...,0.052120,0.053050,0.053050,0.052166,0.053050,0.053050,0.053050,0.052767,0.000414,59
96,6.859402,0.086436,0.295184,0.034187,None,True,10,0,0.01,0.01,...,0.068905,0.053050,0.053050,0.053050,0.059240,0.052166,0.052166,0.055064,0.005007,46
97,8.454790,0.161533,0.284837,0.023137,None,False,10,0.1,2,1,...,0.083922,0.094607,0.090186,0.088417,0.085765,0.088417,0.082228,0.085736,0.005497,25
98,6.959980,0.105371,0.312499,0.038906,None,True,50,0.1,0.01,1,...,0.052120,0.053050,0.053050,0.053050,0.053050,0.053050,0.053050,0.052855,0.000368,53


##### Use the pre-processing pipe to pre-process both the train and test data separately, then fit a model with the above best parameters to the train data and evaluate the result on the test data

In [62]:
twentyRndForPreProcessingPipe.fit(twenty_train.data)
X_train_RndFor_twenty = twentyRndForPreProcessingPipe.transform(twenty_train.data)
X_test_RndFor_twenty = twentyRndForPreProcessingPipe.transform(twenty_test.data)
twentyRndForDefault = RandomForestClassifier()
twentyRndForDefault.fit(X_train_RndFor_twenty, twenty_train.target)
y_pred_RndFor_twenty_Default = twentyRndForDefault.predict(X_test_RndFor_twenty)
metrics.accuracy_score(twenty_test.target, y_pred_RndFor_twenty_Default)

0.6144450345193839

In [64]:
twentyRndFor = RandomForestClassifier(bootstrap=True, ccp_alpha=1e-05, class_weight='balanced', criterion='gini', max_depth=100, max_features='sqrt', max_leaf_nodes=None, max_samples=0.5, min_impurity_decrease=0.0, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=100, oob_score=False, random_state=None)
twentyRndFor.fit(X_train_RndFor_twenty, twenty_train.target)
y_pred_RndFor_twenty = twentyRndFor.predict(X_test_RndFor_twenty)
metrics.accuracy_score(twenty_test.target, y_pred_RndFor_twenty)

0.6113913967073819

In [65]:
metrics.confusion_matrix(twenty_test.target, y_pred_RndFor_twenty)

array([[113,   4,   3,   0,   1,   3,   6,   4,  25,   4,   4,   6,   4,
          9,  17,  85,   8,   5,   7,  11],
       [  3, 226,  36,  15,  10,  32,   8,   4,  16,   4,   1,   2,  16,
          1,  10,   2,   0,   0,   3,   0],
       [  3,  16, 241,  38,  20,  19,   2,   3,  26,   1,   2,   3,   5,
          2,   7,   0,   3,   1,   2,   0],
       [  0,  14,  43, 219,  30,  11,  13,   2,  18,   4,   0,   4,  27,
          2,   2,   0,   0,   0,   2,   1],
       [  1,   7,  11,  34, 245,   8,  12,   7,  24,   1,   3,   1,  23,
          2,   4,   0,   1,   0,   1,   0],
       [  1,  29,  36,   6,   8, 262,   4,   4,  26,   2,   0,   2,   8,
          0,   5,   0,   0,   0,   2,   0],
       [  1,   4,   4,  26,  16,   1, 278,   6,  22,   3,   3,   1,  11,
          0,   4,   1,   7,   0,   2,   0],
       [  2,   4,   7,   4,   2,   4,  11, 246,  73,   5,   1,   0,  19,
          4,   4,   1,   6,   1,   2,   0],
       [  5,   0,   2,   4,   3,   2,   7,  23, 304,  10,   1,  

In [66]:
print(metrics.classification_report(twenty_test.target, y_pred_RndFor_twenty, target_names=twenty_test.target_names))

                          precision    recall  f1-score   support

             alt.atheism       0.40      0.35      0.37       319
           comp.graphics       0.61      0.58      0.59       389
 comp.os.ms-windows.misc       0.58      0.61      0.59       394
comp.sys.ibm.pc.hardware       0.56      0.56      0.56       392
   comp.sys.mac.hardware       0.67      0.64      0.65       385
          comp.windows.x       0.67      0.66      0.67       395
            misc.forsale       0.70      0.71      0.71       390
               rec.autos       0.67      0.62      0.65       396
         rec.motorcycles       0.35      0.76      0.48       398
      rec.sport.baseball       0.73      0.72      0.73       397
        rec.sport.hockey       0.83      0.84      0.83       399
               sci.crypt       0.79      0.66      0.72       396
         sci.electronics       0.45      0.38      0.41       393
                 sci.med       0.77      0.63      0.69       396
         

# Random Forest Classifier - IBMD Movie Reviews
##### Set up a pre-processing pipeline separately from the main-pipeline so that later we can pre-process before training our final model on the train dataset.  Define the parameters over which to do the GSCV and then perform the GSCV on the set of permutations of those parameters.

In [41]:
imbdRndForPreProcessingPipe = Pipeline([
    ('vect', CountVectorizer(lowercase=True, max_df=0.5, max_features=100000, min_df=2, ngram_range=(1, 2), stop_words='english', strip_accents='ascii')),
    ('tfidf', TfidfTransformer()),
    ('scale', StandardScaler(with_mean=False)),
    ('norm', Normalizer())
])
imbdRndForPipe = Pipeline([
    ('ppp', imbdRndForPreProcessingPipe),
    ('clf', RandomForestClassifier())
])
imbdRndForParam = {
    'clf__n_estimators': [10, 50, 100],
    'clf__criterion': ['gini', 'entropy'],
    'clf__max_depth': [None, 20, 100],
    'clf__min_samples_split': [0.01, 2, 10],
    'clf__min_samples_leaf': [0.01, 1, 10],
    'clf__min_weight_fraction_leaf': [0.0, 0.1],
    'clf__max_features': ['sqrt', 'log2', None],
    'clf__max_leaf_nodes': [None, 100, 1000],
    'clf__min_impurity_decrease': [0.0, 0.1],
    'clf__bootstrap': [True],
    'clf__oob_score': [True, False],
    'clf__random_state': [None],
    'clf__class_weight': ['balanced', None],
    'clf__ccp_alpha': [0.0, 0.00001, 0.01],
    'clf__max_samples': [0.1, 0.5, None]
}
imbdRndForGSCV = RandomizedSearchCV(imbdRndForPipe, imbdRndForParam, n_iter=100, cv=10, n_jobs=-1, verbose=1000)
imbdRndForGSCV = imbdRndForGSCV.fit(imbd_train_data, imbd_train_target)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   16.8s
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done   4 task

[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed:  7.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:  7.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  7.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:  7.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  55 tasks      | elapsed:  7.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  7.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  7.7min
Pickling array (shap

[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed: 14.1min
[Parallel(n_jobs=-1)]: Done 106 tasks      | elapsed: 14.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 107 tasks      | elapsed: 14.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 108 tasks      | elapsed: 14.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 14.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 110 tasks      | elapsed: 14.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 111 tasks      | elapsed: 14.7min
Pickling array (shap

[Parallel(n_jobs=-1)]: Done 159 tasks      | elapsed: 18.7min
Pickling array (shape=(22500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed: 18.7min
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 161 tasks      | elapsed: 19.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed: 19.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 163 tasks      | elapsed: 19.1min
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed: 19.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed: 19.4min
Pickling array (shap

[Parallel(n_jobs=-1)]: Done 213 tasks      | elapsed: 23.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 214 tasks      | elapsed: 23.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 215 tasks      | elapsed: 23.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 216 tasks      | elapsed: 23.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed: 23.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 218 tasks      | elapsed: 23.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 219 tasks      | elapsed: 23.4min
Pickling array (shap

...

[Parallel(n_jobs=-1)]: Done 753 tasks      | elapsed: 91.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 754 tasks      | elapsed: 91.1min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 755 tasks      | elapsed: 91.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 756 tasks      | elapsed: 91.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 757 tasks      | elapsed: 91.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 758 tasks      | elapsed: 91.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 759 tasks      | elapsed: 91.7min
Pickling array (shap

[Parallel(n_jobs=-1)]: Done 807 tasks      | elapsed: 95.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 808 tasks      | elapsed: 95.6min
[Parallel(n_jobs=-1)]: Done 809 tasks      | elapsed: 95.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 810 tasks      | elapsed: 95.6min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 811 tasks      | elapsed: 95.8min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 812 tasks      | elapsed: 95.9min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 813 tasks      | elapsed: 95.9min
Pickling array (shap

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 861 tasks      | elapsed: 105.0min
[Parallel(n_jobs=-1)]: Done 862 tasks      | elapsed: 105.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 863 tasks      | elapsed: 105.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 864 tasks      | elapsed: 105.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 865 tasks      | elapsed: 105.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed: 105.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,),

[Parallel(n_jobs=-1)]: Done 914 tasks      | elapsed: 119.2min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 915 tasks      | elapsed: 128.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 916 tasks      | elapsed: 128.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 917 tasks      | elapsed: 128.7min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 918 tasks      | elapsed: 128.8min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 919 tasks      | elapsed: 129.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 920 tasks      | elapsed: 129.1min
Pickling arra

Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 968 tasks      | elapsed: 136.0min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 969 tasks      | elapsed: 138.3min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 970 tasks      | elapsed: 138.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 971 tasks      | elapsed: 140.4min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 972 tasks      | elapsed: 140.5min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,), dtype=int32).
[Parallel(n_jobs=-1)]: Done 973 tasks      | elapsed: 142.8min
Pickling array (shape=(22500,), dtype=int32).
Pickling array (shape=(2500,),

##### The best score and associated parameters followed by a table of more detailed results

In [42]:
imbdRndForGSCV.best_score_

0.79868

In [43]:
for param_name in sorted(imbdRndForParam.keys()):
    print("%s: %r" % (param_name, imbdRndForGSCV.best_params_[param_name]))

clf__bootstrap: True
clf__ccp_alpha: 1e-05
clf__class_weight: 'balanced'
clf__criterion: 'entropy'
clf__max_depth: None
clf__max_features: 'sqrt'
clf__max_leaf_nodes: None
clf__max_samples: None
clf__min_impurity_decrease: 0.0
clf__min_samples_leaf: 0.01
clf__min_samples_split: 10
clf__min_weight_fraction_leaf: 0.0
clf__n_estimators: 100
clf__oob_score: True
clf__random_state: None


In [44]:
df = pd.DataFrame(imbdRndForGSCV.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__random_state,param_clf__oob_score,param_clf__n_estimators,param_clf__min_weight_fraction_leaf,param_clf__min_samples_split,param_clf__min_samples_leaf,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,15.973566,0.818175,0.898994,0.070838,None,False,10,0.1,10,1,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50000,0.000000,50
1,32.062620,0.800747,0.928331,0.071804,None,True,100,0,2,10,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50000,0.000000,50
2,87.710293,2.538017,0.845360,0.060457,None,False,50,0,2,0.01,...,0.6968,0.6816,0.6732,0.6940,0.6992,0.6580,0.6828,0.68504,0.012010,13
3,14.711339,0.226040,0.932070,0.059118,None,False,100,0.1,10,10,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50000,0.000000,50
4,15.128050,0.068032,0.824253,0.052022,None,False,10,0.1,10,1,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50000,0.000000,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,17.233865,0.300770,0.912801,0.046847,None,True,100,0,10,0.01,...,0.8072,0.8116,0.8004,0.7904,0.7980,0.8012,0.7884,0.79868,0.008164,1
96,14.813895,0.101562,0.734374,0.019763,None,False,10,0.1,10,0.01,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50064,0.001920,49
97,266.177607,2.238757,0.893975,0.051389,None,True,100,0,10,0.01,...,0.6968,0.6828,0.6880,0.6952,0.6984,0.6736,0.6856,0.68880,0.007767,12
98,14.940755,0.075586,0.914061,0.027286,None,False,100,0.1,10,10,...,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.5000,0.50000,0.000000,50


##### Use the pre-processing pipe to pre-process both the train and test data separately, then fit a model with the above best parameters to the train data and evaluate the result on the test data

In [67]:
imbdRndForPreProcessingPipe.fit(imbd_train_data)
X_train_RndFor_imbd = imbdRndForPreProcessingPipe.transform(imbd_train_data)
X_test_RndFor_imbd = imbdRndForPreProcessingPipe.transform(imbd_test_data)
imbdRndForDefault = RandomForestClassifier()
imbdRndForDefault.fit(X_train_RndFor_imbd, imbd_train_target)
y_pred_RndFor_imbd_Default = imbdRndForDefault.predict(X_test_RndFor_imbd)
metrics.accuracy_score(imbd_test_target, y_pred_RndFor_imbd_Default)

0.8518

In [68]:
imbdRndFor = RandomForestClassifier(bootstrap=True, ccp_alpha=1e-05, class_weight='balanced', criterion='entropy', max_depth=None, max_features='sqrt', max_leaf_nodes=None, max_samples=None, min_impurity_decrease=0.0, min_samples_leaf=0.01, min_samples_split=10, min_weight_fraction_leaf=0.0, n_estimators=100, oob_score=True, random_state=None)
imbdRndFor.fit(X_train_RndFor_imbd, imbd_train_target)
y_pred_RndFor_imbd = imbdRndFor.predict(X_test_RndFor_imbd)
metrics.accuracy_score(imbd_test_target, y_pred_RndFor_imbd)

0.80516

In [69]:
metrics.confusion_matrix(imbd_test_target, y_pred_RndFor_imbd)

array([[ 9782,  2718],
       [ 2153, 10347]], dtype=int64)

In [70]:
print(metrics.classification_report(imbd_test_target, y_pred_RndFor_imbd))

              precision    recall  f1-score   support

           0       0.82      0.78      0.80     12500
           1       0.79      0.83      0.81     12500

    accuracy                           0.81     25000
   macro avg       0.81      0.81      0.81     25000
weighted avg       0.81      0.81      0.81     25000



# AdaBoost-DecisionTree Classifier - Twenty News Group
##### Set up a pre-processing pipeline separately from the main-pipeline so that later we can pre-process before training our final model on the train dataset.  Define the parameters over which to do the GSCV and then perform the GSCV on the set of permutations of those parameters.

In [ ]:
twentyAdaBooPreProcessingPipe = Pipeline([
    ('vect', CountVectorizer(lowercase=True, max_df=0.5, max_features=10000, min_df=2, ngram_range=(1, 1), stop_words='english', strip_accents='ascii')),
    ('tfidf', TfidfTransformer()),
    ('scale', StandardScaler(with_mean=False)),
    ('norm', Normalizer())
])
twentyAdaBooPipe = Pipeline([
    ('ppp', twentyAdaBooPreProcessingPipe),
    ('clf', AdaBoostClassifier())
])
twentyAdaBooParam = {
    'clf__base_estimator': [twentyDecTre],
    'clf__n_estimators': [25, 50, 100],
    'clf__learning_rate': [0.1, 1.0, 2.5, 10.0],
    'clf__algorithm': ['SAMME.R', 'SAMME'],
    'clf__random_state': [None]
}
twentyAdaBooGSCV = GridSearchCV(twentyAdaBooPipe, twentyAdaBooParam, cv=10, n_jobs=-1, verbose=1000)
twentyAdaBooGSCV = twentyAdaBooGSCV.fit(twenty_train.data, twenty_train.target)

##### The best score and associated parameters followed by a table of more detailed results

In [ ]:
twentyAdaBooGSCV.best_score_

In [ ]:
for param_name in sorted(twentyAdaBooParam.keys()):
    print("%s: %r" % (param_name, twentyAdaBooGSCV.best_params_[param_name]))

In [ ]:
df = pd.DataFrame(twentyAdaBooGSCV.cv_results_)
df

##### Use the pre-processing pipe to pre-process both the train and test data separately, then fit a model with the above best parameters to the train data and evaluate the result on the test data

In [ ]:
twentyAdaBooPreProcessingPipe.fit(twenty_train.data)
X_train_AdaBoo_twenty = twentyAdaBooPreProcessingPipe.transform(twenty_train.data)
X_test_AdaBoo_twenty = twentyAdaBooPreProcessingPipe.transform(twenty_test.data)
twentyAdaBoo = AdaBoostClassifier()
twentyAdaBoo.fit(X_train_AdaBoo_twenty, twenty_train.target)
y_pred_AdaBoo_twenty = twentyAdaBoo.predict(X_test_AdaBoo_twenty)
metrics.accuracy_score(twenty_test.target, y_pred_AdaBoo_twenty)

In [ ]:
metrics.confusion_matrix(twenty_test.target, y_pred_AdaBoo_twenty)

In [ ]:
print(metrics.classification_report(twenty_test.target, y_pred_AdaBoo_twenty, target_names=twenty_test.target_names))

# AdaBoost-DecisionTree Classifier - IBMD Movie Reviews
##### Set up a pre-processing pipeline separately from the main-pipeline so that later we can pre-process before training our final model on the train dataset.  Define the parameters over which to do the GSCV and then perform the GSCV on the set of permutations of those parameters.

In [ ]:
imbdAdaBooPreProcessingPipe = Pipeline([
    ('vect', CountVectorizer(lowercase=True, max_df=0.4, max_features=10000, min_df=3, ngram_range=(1, 3), stop_words='english', strip_accents='ascii')),
    ('tfidf', TfidfTransformer()),
    ('scale', StandardScaler(with_mean=False)),
    ('norm', Normalizer())
])
imbdAdaBooPipe = Pipeline([
    ('ppp', imbdAdaBooPreProcessingPipe),
    ('clf', AdaBoostClassifier())
])
imbdAdaBooParam = {
    'clf__base_estimator': [imbdDecTre],
    'clf__n_estimators': [25, 50, 100],
    'clf__learning_rate': [0.1, 1.0, 2.5, 10.0],
    'clf__algorithm': ['SAMME.R', 'SAMME'],
    'clf__random_state': [None]
}
imbdAdaBooGSCV = GridSearchCV(imbdAdaBooPipe, imbdAdaBooParam, cv=10, n_jobs=-1, verbose=1000)
imbdAdaBooGSCV = imbdAdaBooGSCV.fit(imbd_train_data, imbd_train_target)

##### The best score and associated parameters followed by a table of more detailed results

In [ ]:
imbdAdaBooGSCV.best_score_

In [ ]:
for param_name in sorted(imbdAdaBooParam.keys()):
    print("%s: %r" % (param_name, imbdAdaBooGSCV.best_params_[param_name]))

In [ ]:
df = pd.DataFrame(imbdAdaBooGSCV.cv_results_)
df

##### Use the pre-processing pipe to pre-process both the train and test data separately, then fit a model with the above best parameters to the train data and evaluate the result on the test data

In [ ]:
imbdAdaBooPreProcessingPipe.fit(imbd_train_data)
X_train_AdaBoo_imbd = imbdAdaBooPreProcessingPipe.transform(imbd_train_data)
X_test_AdaBoo_imbd = imbdAdaBooPreProcessingPipe.transform(imbd_test_data)
imbdAdaBoo = AdaBoostClassifier()
imbdAdaBoo.fit(X_train_AdaBoo_imbd, imbd_train_target)
y_pred_AdaBoo_imbd = imbdAdaBoo.predict(X_test_AdaBoo_imbd)
metrics.accuracy_score(imbd_test_target, y_pred_AdaBoo_imbd)

In [ ]:
metrics.confusion_matrix(imbd_test_target, y_pred_AdaBoo_imbd)

In [ ]:
print(metrics.classification_report(imbd_test_target, y_pred_AdaBoo_imbd))